In [24]:
import pandas as pd
import numpy as np
import requests
import io
import re
import datetime
import math
import itertools 

In [25]:
DATA_PATH = "../data/"
urls = {'master': 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'}

In [26]:
# Takes a date from the master file format as a string and convert it to a datetime format
def parse_date(string):
    return datetime.datetime \
                   .strptime(str(re.findall('/([0-9]*)\.', string)[0]),"%Y%m%d%H%M%S")

# Cleans the masterfilelist by droping the empty lines and parsing the dates
def clean_master(master):
    
    # Drops nan columns
    master = master.dropna(how='any')
    
    # Create date columns
    master = master.assign(date=master.url.apply(parse_date))
    
    # Creates a column with the type of the csv file
    master = master.assign(col_type=master.url.apply(lambda url: url.split('.')[-3]))
    
    return master

# Gets the masterfilelist and cleans its content
def fetch_master():
    master = pd.read_csv(urls['master'], sep=" ", header=None, names=['size', 'code', 'url'])
    return clean_master(master)


master = fetch_master()
master.head(7)

,size,code,url,date,col_type
0,150383,297a16b493de7cf6ca809a7cc31d0b93,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:00:00,export
1,318084,bb27f78ba45f69a17ea6ed7755e9f8ff,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:00:00,mentions
2,10768507,ea8dde0beb0ba98810a92db068c0ce99,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:00:00,gkg
3,149211,2a91041d7e72b0fc6a629e2ff867b240,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:15:00,export
4,339037,dec3f427076b716a8112b9086c342523,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:15:00,mentions
5,10269336,2f1a504a3c4558694ade0442e9a5ae6f,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:15:00,gkg
6,149723,12268e821823aae2da90882621feda18,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:30:00,export


In [27]:
# Adds a columns to the master dataframe to easily detect the urls containing the exports, the mentions and the gkg data
def split_master(master):
    splitted = {}
    elements = [['export', 'export'], ['mentions', 'mentions'], ['gkg', 'gkg']]
    for element in elements:
        splitted[element[1]] = master[master.col_type == element[0]]
    return splitted

# Gets subset of data from the master given a time constraint
def sample_master(master, date_start, date_end):
    return master[(master.date >= date_start) & (master.date <= date_end)]

# test
d1 = datetime.datetime.strptime("2015-04-18 22:00:00", "%Y-%m-%d %H:%M:%S")
d2 = datetime.datetime.strptime("2015-04-18 23:00:00", "%Y-%m-%d %H:%M:%S")
split_master(sample_master(master, d1, d2))['export']

,size,code,url,date,col_type
16953,50933,f4d2e1df34f4051ebb4d67b6038c3d28,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:00:00,export
16956,52319,d132eaaa0769f837036d727ecffc5c25,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:15:00,export
16959,57901,fd3008840a3a05b726cbe82c6f379af6,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:30:00,export
16962,66213,c80e0e70422ca8e1bbe2d8d86f2d65fb,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:45:00,export
16965,60673,d6ea37afd2a010761bf5f95fa1cdb39a,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 23:00:00,export


In [38]:
def get_data_columns(event, mentions, gkg):

    col_ex = open(DATA_PATH + event, "r").readlines()[0].rstrip('\n').split(" ")
    col_men = open(DATA_PATH + mentions, "r").readlines()[0].rstrip('\n').split(" ")
    col_gkg = open(DATA_PATH + gkg, "r").readlines()[0].rstrip('\n').split(" ")
    
    return col_ex, col_men, col_gkg

col_ex, col_men, col_gkg = get_data_columns("event_table_name", "mentions_table_name", "gkg_table_name")
# print(col_ex)
# print(col_men)
# print(col_gkg)

col_use_ex, col_use_men, col_use_gkg = get_data_columns("event_table_usefull_name", "mentions_table_usefull_name", "gkg_table_usefull_name")
print(col_use_ex)
print(col_use_men)
print(col_use_gkg)

['GlobalEventID', 'Day', 'MounthYear', 'Year', 'ActionGeo_CountryCode', 'ActionGeo_Lat', 'ActionGeo_Long', 'GoldsteinScale', 'NumMentions']
['GlobalEventId', 'MentionSourceName', 'Confidence', 'MentionDocTone']
['GKGRECORDID', 'DATE', 'Counts', 'SourceCommonName', 'Locations', 'DocumentIdentifier', 'V2Themes', 'Themes', 'V2Tone']


In [29]:
# Cleans the dataframe content by droping the empty lines
def clean_dataframe(data):
    data = data.dropna()
    return data

# Fetches one type of dataframe given the original columns and the wanted columns
def fetch(url,col,col_use):
    data = pd.read_csv(url, sep='\t', names=col, usecols=col_use)
    data = clean_dataframe(data)
    return data


fetch(master.url[0],col_ex,col_use_ex).head(4)

,GlobalEventID,Day,MounthYear,Year,GoldsteinScale,NumMentions,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long
0,410412347,20140218,201402,2014,2.8,5,SF,-30.3098,25.2971
1,410412348,20140218,201402,2014,1.9,5,IN,12.9833,77.5833
2,410412349,20140218,201402,2014,1.9,1,AS,-36.0667,146.4830
3,410412350,20140218,201402,2014,1.9,2,NZ,-41.0000,174.0000


In [30]:
def fetch_all(master, data_type, col, col_use, date_start=None, date_end=None, verbose=True):
    if date_start == None:
        date_start = datetime.datetime(2000, 1, 1)
    if date_end == None:
        date_end = datetime.datetime.now()
    sampled = sample_master(master, date_start, date_end)
    splitted = split_master(sampled)
    all_datas = []
    for datas_url in splitted[data_type]['url']:
        if verbose:
            print(datas_url)
        try:
            datas = fetch(datas_url, col, col_use)
            all_datas.append(datas)
        except:
            if verbose:
                print("Error downloading url %s" % datas_url)
            pass
    return pd.concat(all_datas)


# test
d1 = datetime.datetime.strptime("2015-04-18 21:00:00", "%Y-%m-%d %H:%M:%S")
d2 = datetime.datetime.strptime("2015-04-18 23:00:00", "%Y-%m-%d %H:%M:%S")


# TODO: 1. DROP DUPLICATES IN MENTIONS - 2. MERGE WITH EXPORT - 3. DROP DUPLICATES IN EXPORT
#       4. CHOOSE THE RIGHT COLUMNS

# Duplicate source url check
#fetch_all(master, d1, d2, True) \
#    .groupby('SOURCEURL') \
#    .agg({'Day': 'count'}) \
#    .sort_values('Day', ascending=False) \
#    .head(5)

# Same source url but different actors etc etc
#d = fetch_all(master, d1, d2, True)
#d = d[d.SOURCEURL == 'http://www.theguardian.com/uk-news/2015/apr/18/abubaker-deghayes-brighton-jihadi-syria-amer']
#d

In [31]:
export = fetch_all(master, "export", col_ex, col_use_ex, d1, d2, True)
display(export.head())

http://data.gdeltproject.org/gdeltv2/20150418210000.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418211500.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418213000.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418214500.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418220000.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418221500.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418223000.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418224500.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418230000.export.CSV.zip


,GlobalEventID,Day,MounthYear,Year,GoldsteinScale,NumMentions,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long
1,426527210,20140418,201404,2014,-0.4,2,IT,42.8333,12.83330
2,426527211,20140418,201404,2014,0.0,2,IT,42.8333,12.83330
3,426527212,20150319,201503,2015,3.0,20,ML,12.6500,-8.00000
4,426527213,20150319,201503,2015,-6.5,3,SP,41.3833,2.18333
5,426527214,20150319,201503,2015,-2.0,4,ML,12.6500,-8.00000


In [32]:
mentions = fetch_all(master, "mentions", col_men, col_use_men, d1, d2, True)
display(mentions.head())

http://data.gdeltproject.org/gdeltv2/20150418210000.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418211500.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418213000.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418214500.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418220000.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418221500.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418223000.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418224500.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20150418230000.mentions.CSV.zip


,GlobalEventId,MentionSourceName,Confidence,MentionDocTone
0,426527209,suntimes.com,50,3.846154
1,426527210,themarketbusiness.com,20,-5.230769
2,426527211,themarketbusiness.com,20,-5.230769
3,426400002,time.com,10,-4.536617
4,426388038,arabnews.com,10,-6.984127


In [33]:
gkg = fetch_all(master, "gkg", col_gkg, col_use_gkg, d1, d2, True)
display(gkg.head())

http://data.gdeltproject.org/gdeltv2/20150418210000.gkg.csv.zip
http://data.gdeltproject.org/gdeltv2/20150418211500.gkg.csv.zip
http://data.gdeltproject.org/gdeltv2/20150418213000.gkg.csv.zip
http://data.gdeltproject.org/gdeltv2/20150418214500.gkg.csv.zip
http://data.gdeltproject.org/gdeltv2/20150418220000.gkg.csv.zip
http://data.gdeltproject.org/gdeltv2/20150418221500.gkg.csv.zip
http://data.gdeltproject.org/gdeltv2/20150418223000.gkg.csv.zip
http://data.gdeltproject.org/gdeltv2/20150418224500.gkg.csv.zip
http://data.gdeltproject.org/gdeltv2/20150418230000.gkg.csv.zip


,GKGRECORDID,DATE,SourceCommonName,DocumentIdentifier,Counts,Themes,V2Themes,Locations,V2Tone
0,20150418210000-0,20150418210000,BBC Monitoring,"MAP news agency website, Rabat/BBC Monitoring/...",KILL#21##1#Libya#LY#LY#25#17#LY;,TAX_FNCACT;TAX_FNCACT_REPRESENTATIVE;TAX_FNCAC...,"TAX_FNCACT_PILOT,473;TAX_FNCACT_SPOKESPERSON,6...","1#Libya#LY#LY#25#17#LY;4#Tripoli, Tarabulus, L...","-0.45662100456621,2.28310502283105,2.739726027..."
3,20150418210000-3,20150418210000,oanow.com,http://www.oanow.com/apwire/news/national/arti...,"KILL#168##3#Oklahoma City, Oklahoma, United St...",MANMADE_DISASTER_IMPLIED;KILL;ARMEDCONFLICT;TA...,"TAX_WEAPONS_EXPLOSIVES,1457;MANMADE_DISASTER_I...","2#Oklahoma, United States#US#USOK#35.5376#-96....","-3.74707259953162,1.87353629976581,5.620608899..."
5,20150418210000-5,20150418210000,msn.com,http://www.msn.com/en-ca/news/world/australian...,ARREST#5#Australian teenagers#1#Australia#AS#A...,TAX_ETHNICITY;TAX_ETHNICITY_AUSTRALIAN;ARREST;...,"TAX_FNCACT_DEPUTY,377;TAX_FNCACT_DEPUTY,1760;K...",1#Australia#AS#AS#-27#133#AS;1#Syria#SY#SY#35#...,"-6.03217158176944,1.87667560321716,7.908847184..."
16,20150418210000-16,20150418210000,orlandoadvocate.com,http://www.orlandoadvocate.com/?p=1691,"KILL#8#children lost#3#Orange County, Florida,...",LEADER;TAX_FNCACT;TAX_FNCACT_MAYOR;WB_2433_CON...,"LEGISLATION,1469;TAX_FNCACT_WOMEN,987;TAX_FNCA...","2#Florida, United States#US#USFL#27.8333#-81.7...","-8.15047021943574,2.19435736677116,10.34482758..."
39,20150418210000-39,20150418210000,business-standard.com,http://www.business-standard.com/article/pti-s...,SEIZE#2##1#France#FR#FR#46#2#FR;ARREST#3##1#Un...,TAX_ETHNICITY;TAX_ETHNICITY_FRENCH;TAX_WORLDLA...,"GENERAL_GOVERNMENT,135;GENERAL_GOVERNMENT,721;...",1#United States#US#US#38#-97#US;1#Martinique#M...,"-5.63380281690141,0,5.63380281690141,5.6338028..."


In [34]:
export['GlobalEventID'].nunique()

8291

In [ ]:
my_csv.drop_duplicates(keep='first', inplace=False).shape

In [ ]:
def sample_data(data, step_years, step_months, step_hours, step_minutes):
    years = np.arange(2015,  2018, step_years)
    months = np.arange(1, 12, step_months)
    days = np.arange(1, 31, step_days)
    hours = np.arange(0, 23, step_hours)
    minutes = np.arange(0, 45, step_minutes)
    
    permutations = [
        (d, mo, y, h, mi, 0)
        for y in years
        for mo in months
        for d in days
        for h in hours
        for mi in minutes
    ]
    return data.Date.filter(items=permutations)

In [ ]:
def in_list(x,permutations):
    if x in permutations:
        return x

years = np.arange(2015,2018,1)
months = np.arange(1,12,1)
days = np.arange(1,31,5)
hours = np.arange(0,23,5)
minutes = np.arange(0,45,15)

permutations = [ [d,mo,y,h,mi,0] for y in years for mo in months for d in days for h in hours for mi in minutes]
master.Date.apply(lambda x: in_list(x,permutations))

In [ ]:
for value in list1:
    if len(list2[list2 == value]) == 1:
        list3.append(value)